<a href="https://colab.research.google.com/github/datacraft-paris/2311-Cerisara-LLM/blob/main/LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents
1. [Introduction](https://www.google.com/)
2. [A Brief Overview of LLMs](#paragraph1) (This notebook)
    1. [Background: _Decoder-only_ Language Model](#language_models_are_conditional_probabilities")
    2. [Transformers](#transformers)
    3. [Large Language Model](#llms)
    4. [Exercices](#exercices)


# Background: _Decoder-only_ Language Model (LM) <a name="language_models_are_conditional_probabilities"></a>



Traditionally, a language model (LM) is just a function that assign a probability to a given sequence of words (it can also be letters, tokens, etc).
For a sequence of words $\textbf{s}$, and a trained language model $P_{LM}$, the language model can assign a probability to this sequence:

> $$P(\textbf{s}) = \prod_{i=1}^{|\textbf{s}|} P_{LM}({s_{i}}|s_{<i})$$
where $s_{<i}$ stands for all the previous words.

for example, for the sentence "He runs fast":

> $P$(He) $\times$ $P$(runs|He) $\times$ $P$(fast|he runs)


We can write things like this thanks to the chain rule.\
Indeed we can see this as the probability of predicting the next word at each step, sequentially, from left to right. This is exactly how a language model is trained: it learns to maximize the probability of the _true_ next word $s_{i}$ at each step of the sequence.

A language model is trained on a huge text corpus. If the language model is well trained, we would expect to have meaningful probabilities, for example that $P$("He runs fast") $>$ $P$("He run fast").

This is the standard definition of the language. However, when we use a very deep and large neural network that can model very long range dependencies, the resulting language model can solve interesting tasks without supervision.

One neural network that can model long range dependencies and scale efficiently is the Transformer. Let recall the core component of this neural network.



# Transformers <a name="transformers"></a>

The transformers architecture was first developed for the machine translation task. It was designed to model the long range dependenies between words in a sentence, which is essential for machine translation task.

The main core of the Transformer model is the attention mechanism.

## Attention

There are two types of attention: the self-attention and the cross-attention. The cross-attention models the dependencies between words of two different sequences. This type of attention is used mainly on seq2seq models like [T5](https://huggingface.co/google/flan-t5-base) or [BART](https://huggingface.co/facebook/bart-large)

In the case of _decoder_ large language model, we're interested in the self-attention mechanism. Let see how the self-attention of a sequence $E \in \mathbb{R}^{s * d}$ is computed. $E$ is matrix containing the vectors of each word in the sequence. For example, for the sentence "He runs fast", the matrix could like $[e_{\small He}, e_{\small runs}, e_{\small fast}]$, one vector for each word in the sentence.


---



First, we need to project this input through three matrices: Queries ($W_{q} \in \mathbb{R}^{d * d} $), Keys ($W_{k} \in \mathbb{R}^{d * d}$) and Values ($W_{v} \in \mathbb{R}^{d * d}$)

$$
E_{q} = E \cdot W_{q} \\
E_{k} = E \cdot W_{k} \\
E_{v} = E \cdot W_{v}
$$

$X_{q}, X_{k}, X_{v}$ are all in $\in \mathbb{R}^{s * d}$

Then attention $Attention(E_{q}, E_{k}, E_{v})$ is computed as:
$$
A = softmax(\frac{E_{q} \cdot E_{k}^{T}}{\sqrt{d}})\\
C = A \cdot E_{v}
$$

The attention scores are the square matrix $A \in \mathbb{R}^{s*s} $, where each $A_{i,j}$ represents the attention score (between 0 and 1) of word $i$ on word $j$. These scores indicate the relative importance of each word to others in the context of the task. These scores tell us which word is important for which word for solving the task. A self-attention matrix of a sentence "he runs fast" could look like that:

<p align="center">
  <img src="https://github.com/datacraft-paris/2311-Cerisara-LLM/blob/main/illustration/attention.png?raw=true:, width=500" alt="attention" width=500 class="center">
</p>

Each $A_{i,j}$ is a probability computed by normalizing the scores in the columns.

The tokens $E_{v}$ (values) are then sum weighted by these attention scores in order to get the ouput $C \in \mathbb{R}^{s * d}$. This $C$ contains tokens vectors that are **contextualized**, meaning that each token has in itself the information about all the other tokens in the sequence.

During training, the model learns to assign relevant attention scores to each tokens and to produce meaningful contextual vectors $C$ for solving the task.

## Multihead attention

What's new in the _Attention is all you need_ paper is not attention but multihead attention. This consists in concatenating several independent attention operations:

$$
MultiHeadAttention(E_{q}, E_{k}, E_{v}) = Concat(C_{1}, C_{2}, ..., C_{n})
$$

where each $C_{i}$ is an independant attention computation:
$$
C_{i} = Attention(E_{q}, E_{k}, E_{v})
$$

Having multiple heads will help the model to have multiple "views" on the input sentence.

## Transformer layer
$$
\begin{aligned}
\text{TRANSFORMER}(E) &= E_{q},E_{k},E_{v} = (E \cdot W_{q}), (E \cdot W_{k}), (E \cdot W_{v}) \\
H &= \text{MultiHeadAttention}(E_{q},E_{k},E_{v}) \\
H_{(z)} &= \text{LayerNorm}(H + E) \\
F &= \text{MLP}(H_{(z)}) \\
C &= \text{LayerNorm}(F + H_{(z)}) \\
\end{aligned}
$$

The transformer takes as input a sequence of vectors $E \in \mathbb{R}^{s*d}$, and also outputs a sequence of vectors $C \in \mathbb{R}^{s*d}$. So what is the difference between the two sequences ? The difference is that the first sequence of vectors consist of vectors that are independent of each other. However, the vectors in the second sequence are contextualized, meaning each vector (token) incorporates information about all other tokens in the sequence. So the Transformer litterally _transforms_ the input sequence, and this transformation is a contextualization.





# LLM = LM trained with *HUGE* Transformers on *HUGE* text corpora. <a name="llms"></a>

The transformer takes as inputs a $X$ sequence of vectors and output $C$ also a sequence of vectors. As such, this is not a language model, but just a neural network that can be used on any task / domain (computer vision, speech processing, etc). To perform language modeling, we add a matrix $W \in \mathbb{R}^{d * |V|}$ on top of the transformer's output. This matrix contains one vector for each word (or token) seen in the training set. Given an output vector $\textbf{c}$ of the transformer, the probability of a given word $v$ in the vocabulary $V$ is computed as:

$$
P_{LM}(y = v|\textbf{c}) = \frac{W_{v} \cdot c}{\sum\limits_{v' \in V}W_{v'} \cdot c}
$$

For example, let's say we have the input sentence "He runs fast" that got forward into the transformer, giving the output $C=[\textbf{c}_{\small He}, \textbf{c}_{\small runs}, \textbf{c}_{\small fast}]$. The probability that "fast" follows the word "runs" is computed as:

$$
P_{LM}(fast|\textbf{c}_{\small runs}; W) = \frac{W_{fast} \cdot \textbf{c}_{\small runs}}{\sum\limits_{v' \in V}W_{v'} \cdot \textbf{c}_{\small runs}}
$$

This is the language modelling objective applied to a tranformer. Usually, the transformer language model is trained on a huge quantity of text corpora $D$ chunked into long sequences $s$ (for example Llama is trained on sequences of 4096 tokens).

Training the transformer language model is achieved using the maximum likelihood of training corpora $D$:

>$$
\underset{\theta}{\mathrm{argmax}} \prod_{s \in D} \prod_{i=1}^{|\textbf{s}|} P_{LM}({s_{i}}|s_{<i}; \theta)
$$
where \theta is the parameter of the LLM: the matrix $W$ and all the trainable matrices in the transformer model.

When the training corpora is enough huge (>1T tokens) and the transformer contains enough parameters (>7B parameters), the language model learns to perform well on different language tasks (translation, classification, summarization, coding, etc.)

# Exercices <a name="exercices"></a>

## Dissecting an LLM

1. Download the StableLM 3B and print the model.

2. Identify the parameters of the attention module. What are their dimensions ?

3. Identify the paramaters of the MLP module. What are their dimensions ?

4. Identify the language modelling matrix, what is its dimension ?

5. Get the tokenizer associated with the LLM. Tokenize the sentence "He runs" and forward it into the LLM in order to get the last hidden states. What is the dimensions ?

6. Given the last hidden state, compute the probability of having the word "fast" as the next word.

# The numer of parameters of the LLM

1. What is exactly the number of paramaters the model ?

2. How many parameters have the attention module ? and for MLP module ? In terme of percentage ?